In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from PIL import Image
import numpy as np
from tqdm import tqdm

In [ ]:
# load_data = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/train_image.npy')
# load_label = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/train_label.npy')
# load_test = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/test_image.npy')

load_data = np.load('/content/drive/MyDrive/AI_Lecture/(new)[2021 Spring Class]Ajou AI by JBR/train_image.npy')
load_label = np.load('/content/drive/MyDrive/AI_Lecture/(new)[2021 Spring Class]Ajou AI by JBR/train_label.npy')
load_test = np.load('/content/drive/MyDrive/AI_Lecture/(new)[2021 Spring Class]Ajou AI by JBR/test_image.npy')

# print(load_data.shape)

print(load_label.shape)
print(load_test.shape)

(895,)
(500, 256, 256, 3)


In [ ]:
# 구축된 .npy파일을 Pytorch DataLoader을 사용할 수 있도록 CUSTOM DATASET을 만듬.
import numpy as np

class CUB200(data.Dataset):
    def __init__(self, Train, transform = None):
        super(CUB200, self).__init__()
        """
        Train : bool = True
        """
        if Train == True:
            load_data = np.load('/content/drive/MyDrive/AI_Lecture/(new)[2021 Spring Class]Ajou AI by JBR/train_image.npy')
            load_label = np.load('/content/drive/MyDrive/AI_Lecture/(new)[2021 Spring Class]Ajou AI by JBR//train_label.npy')
            self.image = load_data[:700:]
            self.label = load_label[:700:]
            self.transform = transform
            
        else:
            load_data = np.load('/content/drive/MyDrive/AI_Lecture/(new)[2021 Spring Class]Ajou AI by JBR/train_image.npy')
            load_label = np.load('/content/drive/MyDrive/AI_Lecture/(new)[2021 Spring Class]Ajou AI by JBR//train_label.npy')
            self.image = load_data[700::]
            self.label = load_label[700::]
            self.transform = transform

    def __getitem__(self, index):
        img, target = self.image[index], self.label[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.image)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# train_data에만 data augmentaion을 적용함.
transform_train = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        # transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transform_test = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
# CUSTOM DATASET을 이용하여 train_loader, test_loader을 구축.

batch_size = 64

train_loader = torch.utils.data.DataLoader(
    dataset = CUB200(True, transform = transform_train),
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = CUB200(False, transform = transform_test),
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
def training_model(model, criterion, optimizer, scheduler, num_epochs = 25):
    for epoch in range(num_epochs):
        scheduler.step()

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 9 == 0:
                print('[%d, %5d] loss: %.7f' %
                    (epoch + 1, (i + 1), running_loss / 20))
                running_loss = 0.0
        
        train_correct = 0
        train_total = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.squeeze()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        print('[%d epoch] Accuracy of the network on the train images: %d %%' %
              (epoch + 1, 100 * train_correct / train_total))
        
    print("End Training do it eval_accuracy")
    return model

In [ ]:
def eval_accuracy(model):
    class_correct = list(0. for i in range(50))
    class_total = list(0. for i in range(50))

    correct = 0
    total = 0

    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images = images.squeeze()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
                    
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * correct / total), correct, total)            
                
    return 

In [ ]:
# resnet18 = models.resnet18(pretrained=True) 
resnet101 = models.resnet101(pretrained=True)  # epoch 15 => 74 %
resnet152 = models.resnet152(pretrained=True)
resnet50 = models.resnet50(pretrained=True)
# alexnet = models.alexnet(pretrained=True)  # epoch 15 => 35%
# squeezenet = models.squeezenet1_0(pretrained=True)  # epoch 15 => 16%
# vgg16 = models.vgg16(pretrained=True)  # epoch 15 => 27%
# densenet = models.densenet161(pretrained=True)  # epoch 5 => 63%
# inception = models.inception_v3(pretrained=True)  # kernel error
# googlenet = models.googlenet(pretrained=True)  # epoch 15 => 38%  => epoch 5 로 바꿔서 48% not bad
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True)  # epoch 5 => 44%
# mobilenet_v2 = models.mobilenet_v2(pretrained=True) # epoch 5 => 44 %
# mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)  # epoch 10 => 31%
# mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)  # epoch 10 => 21%
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)  # epoch 10 => 74%
resnext101_32x8d = models.resnext101_32x8d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)  # epoch 10 => 34%
wide_resnet101_2 = models.wide_resnet101_2(pretrained=True)
# mnasnet = models.mnasnet1_0(pretrained=True)  # epoch 10 => 28%


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth


In [ ]:
# model_ft = models.mnasnet1_0(pretrained=True)
model_ft = wide_resnet101_2
for param in model_ft.parameters():
    param.requires_grad = False

# print(model_ft)

In [ ]:
num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 50)
model_ft.fc = nn.Sequential(
    nn.Linear(num_ftrs, 50),
    # nn.ReLU(),
    # nn.Dropout(0.2),
    # nn.Linear(500, 50),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)
# print(model_ft)

In [ ]:
"""For Other Net"""
# model_ft.add_module('lastlayer', module=nn.Linear(1000,50))
# print(model_ft)

In [ ]:
num_epochs = 10
model_ft.to(device)
# criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()
# optimizer = optim.SGD(model_ft.parameters(), lr = 0.001, momentum = 0.9)
optimizer = optim.Adam(model_ft.parameters())
# lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1)

In [ ]:
model_ft = training_model(model_ft, criterion, optimizer, lr_scheduler, num_epochs)

In [ ]:
eval_accuracy(model_ft)

In [ ]:
class TestSet(data.Dataset):
    def __init__(self, transform = None):
        super(TestSet, self).__init__()
        load_test = np.load('/content/drive/MyDrive/Colab Notebooks/AI/(new)[2021 Spring Class]Ajou AI by JBR/test_image.npy')
        self.image = load_test[::]
        self.transform = transform
            

    def __getitem__(self, index):
        img = self.image[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.image)

ac_transform_test = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

ac_test_loader = torch.utils.data.DataLoader(
    dataset = TestSet(transform = ac_transform_test),
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
ans = []
with torch.no_grad():
    for i, data in enumerate(ac_test_loader):
        data = data.squeeze()
        data = data.to(device)
        outputs = model_ft(data)
        _, predicted = torch.max(outputs, 1)
        for p in predicted:
            ans.append(p.item())
        # print(predicted)

print(ans)

In [ ]:
result_txt = list_data = [str(int(a)).strip('\n\r') for a in ans]
print(result_txt)
print(len(result_txt))

In [ ]:
pip install pycryptodomex --no-binary :all:

In [ ]:
import json
from base64 import b64encode
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad

def read_txt(fileName):
    with open(fileName, 'rt') as f:
        list_data = [a.strip('\n\r') for a in f.readlines()]
    return list_data

def write_json(fileName, data):
    with open(fileName, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_key(key_path):
    with open(key_path, "rb") as f:
        key = f.read()
    return key

def encrypt_data(key_path, ans_list, encrypt_store_path='ans.json'):
    key = load_key(key_path)
    data = " ".join([str(i) for i in ans_list])
    encode_data = data.encode()
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(encode_data, AES.block_size))
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    write_json(encrypt_store_path, {'iv':iv, 'ciphertext':ct})

if __name__=="__main__":
    # 1.이메일을 통해서 전달 받은 키 파일의 경로 입력
    key_path = "/content/drive/MyDrive/Colab Notebooks/AI/team12.pem"
    # 2. 예측한 결과를 텍스트 파일로 저장했을 경우 리스트로 다시 불러오기
    # 본인이 원하는 방식으로 리스트 형태로 예측 값을 불러오기만 하면 됨(순서를 지킬것)
    #raw_ans_path = "ans.txt"
    #ans = read_txt(raw_ans_path)
    ans = result_txt
    # 3. 암호화된 파일을 저장할 위치
    encrypt_ans_path = "/content/drive/MyDrive/Colab Notebooks/AI/resultfile/final2.json"
    # 4. 암호화!(pycrytodome 설치)
    encrypt_data(key_path, ans, encrypt_ans_path)